**Setup the application**

In [16]:
import glob, sys, threading

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from grams.prelude import GRAMS, ROOT_DIR, DATA_DIR, I, fetch_tables, WikidataSemanticModelHelper

from sm_widgets.widgets.annotator import Annotator, BatchAnnotator
from sm_widgets_integration.wikidata import get_qnode_db, get_ontclass_db, get_ontprop_db
from sm_widgets_integration.with_grams import convert_table
from sm_widgets.widgets.slider import DisplayShell, Slider
from sm.prelude import M

In [2]:
cwd = ROOT_DIR / "examples/misc"
cfg = OmegaConf.load(ROOT_DIR / "grams.yaml")
grams = GRAMS(DATA_DIR, cfg)

**Fetch tables from a webpage and select table that we want to annotate**

In [3]:
tables = fetch_tables("https://en.wikipedia.org/wiki/President_of_the_National_Council_(Austria)")
# tables = fetch_tables("https://en.wikipedia.org/wiki/Novartis")

In [4]:
slider = Slider(app=DisplayShell(render_fn=lambda df: display(df)), app_render_fn='set_data', embedded_error_log=True)
slider.render()

threading.Thread(
    target=lambda: slider.set_data([
        {"description": "<br>" + " > ".join([r.header for r in table.context]), "args": [table.as_df()]}
        for table in tables
    ])
).start()

HTML(value='<div id="79785be8-895d-4c61-b8d9-17ab86465abb"></div>')

Output()

<IPython.core.display.Javascript object>

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='c05c0327-1aae-4fcc-bf2b-2c3bb7b7f8b6')

Output()

<IPython.core.display.Javascript object>

**Annotate the selected table**

In [9]:
table.id

'https://en.wikipedia.org/wiki/President_of_the_National_Council_(Austria)/10'

In [5]:
selected_table = 10

table = tables[selected_table]
table = table.as_relational_linked_table(table_id=f"{table.page_url}/{selected_table}")
annotation = grams.annotate(table)

save table data (optional)

**Show the annotation**

In [12]:
assistant = None
annotator = Annotator(
    entities=get_qnode_db(grams.qnodes), 
    ontclasses=get_ontclass_db(grams.wdclasses), 
    ontprops=get_ontprop_db(grams.wdprops), 
    prop_type_uri='http://www.wikidata.org/prop/P31', 
    savedir=cwd / "ground-truth", 
    eshost='http://mira.isi.edu:9200', 
    dev=False, 
    assistant=assistant)
annotator.render(True)

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='49020d53-6da3-49c2-803c-2651432efb1b')

HTML(value='<div id="eb56dd6d-c3cc-470c-a5df-faa4fa55e6c1"></div>')

<IPython.core.display.Javascript object>

In [13]:
threading.Thread(
    target=lambda: annotator.set_data(table.get_friendly_fs_id(), convert_table(table), sms=[annotation.sm])
).start()